In [73]:
import numpy as np
from scipy import special

In [80]:
class NeuralNetwork:
    """
    A simple implementation of a Neural Network.

    Attributes:
        inodes (int): The number of input nodes.
        hnodes (int): The number of hidden nodes.
        onodes (int): The number of output nodes.
        lr (float): The learning rate.
        wih (numpy.ndarray): The weights between the input and hidden layers.
        who (numpy.ndarray): The weights between the hidden and output layers.
        activation_function (function): The activation function to use.

    🤖 prompt: generate docs
    """
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes

        # Weight sampling rule of thumb (page 103 book DLO): 
        #   - normal distribution: mean zero
        #   - standard deviation: inverse of the square root of the number of links into a node
        self.wih = np.random.normal(0.0, pow(self.hnodes,0.5), (self.hnodes, self.inodes))  # weights from input to hidden layer
        self.who = np.random.normal(0.0, pow(self.onodes,0.5), (self.onodes, self.hnodes))  # weights from hidden to output layer
        
        self.lr = learningrate
        self.activation_function = lambda x: special.expit(x)   # sigmoid function

    def train(self, inputs_list, targets_list):
        """
        Trains the neural network on a single batch of inputs and targets.

        Args:
            inputs_list (list): A list of input values for a single batch of training examples.
            targets_list (list): A list of target values for a single batch of training examples.

        Returns:
            None

        🤖 prompt: generate docs
        """
        # inputs_list = [1, 2, 3]
        # inputs_array = [[1 2 3]]
        # inputs_transposed = [[1]
        #                      [2]
        #                      [3]]
        inputs = np.array(inputs_list, ndmin=2)                     # convert to 2d for matrix multiplication
        inputs_transposed = inputs.T                                # transpose to ensure same orientation as weights (column vector)
        targets = np.array(targets_list, ndmin=2)                   # convert to 2d for matrix multiplication
        targets_transposed = targets.T                              # transpose to ensure same orientation as weights (column vector)
        
        hidden_inputs = self.wih @ inputs_transposed                # calculate signals into hidden layer, using matrix multiplication
        hidden_outputs = self.activation_function(hidden_inputs)    # pass through activation function to calculate final values
        
        final_inputs = self.who @ hidden_outputs                    # calculate signals into output layer, using matrix multiplication
        final_outputs = self.activation_function(final_inputs)      # pass through activation function to calculate final values
        
        output_errors = targets_transposed - final_outputs          # output layer is the final layer, so error is (target - actual)
        hidden_errors = self.who.T @ output_errors                  # errors propagated to the hidden layer
        
        # update the weights for the links between the hidden and output layers
        self.who += self.lr * (output_errors * final_outputs * (1.0 - final_outputs)) @ hidden_outputs.T
        
        # update the weights for the links between the input and hidden layers
        self.wih += self.lr * (hidden_errors * hidden_outputs * (1.0 - hidden_outputs)) @ inputs
        pass
    
    def query(self, inputs_list):
        """
        Given an input list, returns the output of the neural network.

        Args:
            inputs_list (list): A list of input values for the neural network.

        Returns:
            numpy.ndarray: A 2D numpy array containing the output values of the neural network.

        🤖 prompt: generate docs
        """
        # inputs_list = [1, 2, 3]
        # inputs_array = [[1 2 3]]
        # inputs_transposed = [[1]
        #                      [2]
        #                      [3]]
        inputs_array = np.array(inputs_list, ndmin=2)               # convert to 2d for matrix multiplication
        inputs_transposed = inputs_array.T                          # transpose to ensure same orientation as weights (column vector)
        
        hidden_inputs = self.wih @ inputs_transposed                # calculate signals into hidden layer, using matrix multiplication
        hidden_outputs = self.activation_function(hidden_inputs)    # pass through activation function to calculate final values
        
        final_inputs = self.who @ hidden_outputs                    # calculate signals into output layer, using matrix multiplication
        final_outputs = self.activation_function(final_inputs)      # pass through activation function to calculate final values
        
        return final_outputs

In [81]:
input_nodes = 3
hidden_nodes = 1
output_nodes = 3
learning_rate = 0.1

nn = NeuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

# In supervised learning, we would normally use many input-target pairs
# and use each pair to update the network weights.
inputs = [1.0, 0.5, -1.5]
targets = [0.5, 0.5, 0.5]
nn.train(inputs, targets)

outputs = nn.query(inputs)
outputs

array([[0.33728861],
       [0.50326998],
       [0.67109129]])